In [68]:
import numpy as np
import cv2
from os import startfile

from ffpyplayer.player import MediaPlayer

In [69]:
face_detector = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')

In [84]:
def play_video(path):
    frame_name = 'Video'
    vid = cv2.VideoCapture(path)
#     audio = MediaPlayer(path)
    
    cv2.namedWindow(frame_name, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(frame_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    fps = vid.get(cv2.CAP_PROP_FPS)
    print('Video fps:', fps)
    
    while vid.isOpened():
        ret, frame = vid.read()
#         audio_frame, val = audio.get_frame()
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face = face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(30,30), maxSize=(200,200))
        for (x, y, w, h) in face:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        if not ret:
            print('End of video...')
            break
        cv2.imshow(frame_name, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
#         if val != 'eof' and audio_frame is not None:
#             img, t = audio_frame
        
    vid.release()
    cv2.destroyAllWindows()

In [85]:
play_video('data/rcpc_loveisover_mcountdown.webm')

Video fps: 29.97002997002997
